In [63]:
import mysql.connector
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, Column, String, Float, DateTime, Integer
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import declarative_base

In [64]:
# Configurar la conexión a la base de datos
user = "root"
password = "12345678"
host = "localhost"
database = "cripto_db"

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [65]:
#configurar pandas para que muestre los numeros completos y con solo 2 decimales
pd.set_option('display.float_format', '{:.2f}'.format)

In [66]:
df_spread = pd.read_csv('../Dataset/spread_binance.csv')

In [67]:
df_spread.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14227 entries, 0 to 14226
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Par de Trading  14227 non-null  object 
 1   Fecha           14227 non-null  object 
 2   open            14227 non-null  float64
 3   high            14227 non-null  float64
 4   low             14227 non-null  float64
 5   close           14227 non-null  float64
dtypes: float64(4), object(2)
memory usage: 667.0+ KB


In [68]:
# Cambiar los nombres de las columnas
df_spread.rename(columns={'Par de Trading': 'nombre','Fecha': 'fecha'}, inplace=True)

In [69]:
df_spread.head(10)

,nombre,fecha,open,high,low,close
0,BTCUSDT,2018-01-01 09:14:15.655323,13715.65,13818.55,12750.00,13380.00
1,BTCUSDT,2018-01-02 09:14:15.655323,13382.16,15473.49,12890.02,14675.11
2,BTCUSDT,2018-01-03 09:14:15.655323,14690.00,15307.56,14150.00,14919.51
3,BTCUSDT,2018-01-04 09:14:15.655323,14919.51,15280.00,13918.04,15059.54
4,BTCUSDT,2018-01-05 09:14:15.655323,15059.56,17176.24,14600.00,16960.39
5,BTCUSDT,2018-01-06 09:14:15.655323,16960.39,17143.13,16011.21,17069.79
6,BTCUSDT,2018-01-07 09:14:15.655323,17069.79,17099.96,15610.00,16150.03
7,BTCUSDT,2018-01-08 09:14:15.655323,16218.85,16322.30,12812.00,14902.54
8,BTCUSDT,2018-01-09 09:14:15.655323,14902.54,15500.00,14011.05,14400.00
9,BTCUSDT,2018-01-10 09:14:15.655323,14401.00,14955.66,13131.31,14907.09


In [70]:
# Crear una sesión
Session = sessionmaker(bind=engine)
session = Session()

# Definir la clase modelo para la tabla "spread_cripto"
Base = declarative_base()

class SpreadCripto(Base):
    __tablename__ = 'spread_cripto'

    id = Column(Integer, primary_key=True)
    fecha = Column(DateTime)
    nombre = Column(String)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)


# Leer los datos desde un DataFrame y cambiar formato de columna fecha
df_spread = pd.DataFrame(df_spread)
df_spread['fecha'] = df_spread['fecha'].apply(lambda x: pd.to_datetime(x).date())


# Recorrer los datos y cargarlos en la base de datos
for index, row in df_spread.iterrows():
    entry = SpreadCripto(
        fecha=row['fecha'], 
        nombre=row['nombre'], 
        open=row['open'],
        high=row['high'],
        low=row['low'],
        close=row['close']
    )
    session.add(entry)

# Confirmar los cambios y cerrar la sesión
session.commit()
session.close()

print('Datos importados a la tabla spread_cripto')


Datos importados a la tabla spread_cripto
